<div style="background: #000;
            color: #FFF;
            margin: 0px;
            padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 18 Lab</h1>
</div>

For today's lab, we'll be using the Marvel comic book characters data set available from [538](https://github.com/fivethirtyeight/data/tree/master/comic-characters)

The data comes from [Marvel Wikia](http://marvel.wikia.com/Main_Page) and [DC Wikia](http://dc.wikia.com/wiki/Main_Page). Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: `dc-wikia-data.csv` and `marvel-wikia-data.csv`. Each file has the following variables:

Variable | Definition
---|---------
`page_id` | The unique identifier for that characters page within the wikia
`name` | The name of the character
`urlslug` | The unique url within the wikia that takes you to the character
`ID` | The identity status of the character (Secret Identity, Public identity, [on marvel only: No Dual Identity])
`ALIGN` | If the character is Good, Bad or Neutral
`EYE` | Eye color of the character
`HAIR` | Hair color of the character
`SEX` | Sex of the character (e.g. Male, Female, etc.)
`GSM` | If the character is a gender or sexual minority (e.g. Homosexual characters, bisexual characters)
`ALIVE` | If the character is alive or deceased
`APPEARANCES` | The number of appareances of the character in comic books (as of Sep. 2, 2014. Number will become increasingly out of date as time goes on.)
`FIRST APPEARANCE` | The month and year of the character's first appearance in a comic book, if available
`YEAR` | The year of the character's first appearance in a comic book, if available


Importing the libraries we'll be using

In [3738]:
import numpy as np 
import pandas as pd 
import datetime

In [3739]:
#Increase column width 
pd.options.display.max_colwidth = 100 #'Spider-Man (Peter Parker)'

Let's open the `marvel-wikia-data.csv` file and generate a dataframe. Then we'll check the top of the `df` to see what we have.

In [3740]:
df = pd.read_csv("../datasets/marvel-wikia-data.csv")
df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


As you can see we have a couple of issues:
* The column `names` has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for `name` to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column `origin` for the superhero's place of origin (usually Earth-616).
* The `urlslug` column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the `Spider-Man_(Peter_Parker)` from the `urlslug` (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.
* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).
* Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.
* Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 
* Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)
* Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.
* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

Fix as many of these issues as you're able to. At the minimum, complete 4 of the above bulletpoints.

In [3741]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   ID                12606 non-null  object 
 4   ALIGN             13564 non-null  object 
 5   EYE               6609 non-null   object 
 6   HAIR              12112 non-null  object 
 7   SEX               15522 non-null  object 
 8   GSM               90 non-null     object 
 9   ALIVE             16373 non-null  object 
 10  APPEARANCES       15280 non-null  float64
 11  FIRST APPEARANCE  15561 non-null  object 
 12  Year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB


In [3742]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories('name')

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


* The column names has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for name to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column origin for the superhero's place of origin (usually Earth-616).

In [3743]:
# df["SecretIdentity1"] = df['name'].str.split('(',1).str[1].str.split(")",2).str[-3]
df["SecretIdentity"] = df['name'].str.split('(',-1, False).str[1].str.split(')',3).str[-2]

# dropping null value columns to avoid errors 
#df.dropna(inplace = True) 
   
# new data frame with split value columns 
#data["Team"]= data["Team"].str.split("t", n = 1, expand = True) 

df[df["SecretIdentity"].notnull()]
#df["SecretIdentity"] = df["SecretIdentity"].fillna(df["SecretIdentity2"])
df["SecretIdentity"]

0                   Peter Parker
1                  Steven Rogers
2        James \"Logan\" Howlett
3         Anthony \"Tony\" Stark
4                   Thor Odinson
                  ...           
16371                  Earth-616
16372                Thanos' son
16373                     Skrull
16374                 Spiderling
16375                  Earth-616
Name: SecretIdentity, Length: 16376, dtype: object

In [3744]:
df["SecretIdentity"] = df["SecretIdentity"].replace(['Earth-616'],"NA")
df["SecretIdentity"] = df["SecretIdentity"].replace(['Earth-7888'],"NA")
df["SecretIdentity"] = df["SecretIdentity"].replace(['Mania'],"NA")
df[df["SecretIdentity"].notnull()]

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year,SecretIdentity
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0,Peter Parker
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0,Steven Rogers
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0,"James \""Logan\"" Howlett"
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0,"Anthony \""Tony\"" Stark"
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0,Thor Odinson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach (Earth-616),\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN,NA
16372,665474,Thane (Thanos' son) (Earth-616),\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Thanos' son
16373,695217,Tinkerer (Skrull) (Earth-616),\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Skrull
16374,708811,TK421 (Spiderling) (Earth-616),\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN,Spiderling


In [3745]:
df["origin"] = df['name'].str.split("(",3).str[2].str.split(")",1).str[0]
#df['name'].str.split('(').str[1].replace(")","")
df["origin"]

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
16371          NaN
16372    Earth-616
16373    Earth-616
16374    Earth-616
16375          NaN
Name: origin, Length: 16376, dtype: object

In [3746]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories("origin")

,page_id,APPEARANCES,Year
origin,,,
Earth-616,794183741,14846.0,4395394.0
Earth-7888,62360,0.0,1993.0
Mania,670753,23.0,2003.0


In [3747]:
## Superhero Name. I like Ironman
df["name"] = df['name'].str.split('(',3).str[0]
print(df["name"][3])

Iron Man 


In [3748]:
#To reorganize the columns
df.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'Year',
       'SecretIdentity', 'origin'],
      dtype='object')

In [3749]:
df = df.reindex(['page_id',
 'name',
 'SecretIdentity',
 'origin',
 'urlslug',
 'ID',
 'ALIGN',
 'EYE',
 'HAIR',
 'SEX',
 'GSM', 
 'ALIVE',
 'APPEARANCES',
 'FIRST APPEARANCE',
 'Year'], axis=1)

df

,page_id,name,SecretIdentity,origin,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man,Peter Parker,NaN,\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America,Steven Rogers,NaN,\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,Wolverine,"James \""Logan\"" Howlett",NaN,\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,Iron Man,"Anthony \""Tony\"" Stark",NaN,\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor,Thor Odinson,NaN,\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,NA,NaN,\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16372,665474,Thane,Thanos' son,Earth-616,\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer,Skrull,Earth-616,\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16374,708811,TK421,Spiderling,Earth-616,\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


* The urlslug column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/PeterParker(Earth-616) but if you use the Spider-Man_(Peter_Parker) from the urlslug (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.

In [3750]:

#changint the display so that the link appears complete and we can click it to open in browser
#df["urlslug"]= df["urlslug"].str.split("_").str[0]
df["urlslug"] = "http://marvel.fandom.com/wiki/" + df["urlslug"].str.split("/").str[1].replace(["   "], "")
#df["urlslug"]= df["urlslug"].str[:][1]
df["urlslug"]
df


,page_id,name,SecretIdentity,origin,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man,Peter Parker,NaN,http://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America,Steven Rogers,NaN,http://marvel.fandom.com/wiki/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,Wolverine,"James \""Logan\"" Howlett",NaN,http://marvel.fandom.com/wiki/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,Iron Man,"Anthony \""Tony\"" Stark",NaN,http://marvel.fandom.com/wiki/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor,Thor Odinson,NaN,http://marvel.fandom.com/wiki/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,NA,NaN,http://marvel.fandom.com/wiki/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16372,665474,Thane,Thanos' son,Earth-616,http://marvel.fandom.com/wiki/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer,Skrull,Earth-616,http://marvel.fandom.com/wiki/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,NaN,Living Characters,NaN,NaN,NaN
16374,708811,TK421,Spiderling,Earth-616,http://marvel.fandom.com/wiki/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,NaN,Living Characters,NaN,NaN,NaN


* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).

In [3751]:
df["ALIGN"] = df["ALIGN"].str.split('C',1).str[0]
df["ALIGN"] 

0           Good 
1           Good 
2        Neutral 
3           Good 
4           Good 
           ...   
16371        Bad 
16372       Good 
16373        Bad 
16374    Neutral 
16375        Bad 
Name: ALIGN, Length: 16376, dtype: object

In [3752]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories("ALIGN")

,page_id,APPEARANCES,Year
ALIGN,,,
Bad,2030768533,52910.0,12780912.0
Good,1310680907,150603.0,8670851.0
Neutral,771906748,41091.0,4122737.0


* Same for the columns EYE, HAIR, SEX and ID. We want to drop the extraneous data and have simple categories (e.g. for ID we want categories "Known","Public", "Secret", "None" and for SEX we just want "Male", "Female","Genderfluid","Agender". There's a way shown below that shows the various possible categories.

In [3753]:
df["EYE"] = df["EYE"].str.split(' ').str[0]
df["EYE"] 

0        Hazel
1         Blue
2         Blue
3         Blue
4         Blue
         ...  
16371    Green
16372     Blue
16373    Black
16374      NaN
16375      NaN
Name: EYE, Length: 16376, dtype: object

In [3754]:
df['HAIR'] = df['HAIR'].str.split(' ').str[0]
df['HAIR']

0        Brown
1        White
2        Black
3        Black
4        Blond
         ...  
16371       No
16372     Bald
16373     Bald
16374      NaN
16375      NaN
Name: HAIR, Length: 16376, dtype: object

In [3755]:
##SEX
df["SEX"] = df['SEX'].str.split(' ').str[0]
df["SEX"] 

0        Male
1        Male
2        Male
3        Male
4        Male
         ... 
16371    Male
16372    Male
16373    Male
16374    Male
16375     NaN
Name: SEX, Length: 16376, dtype: object

In [3756]:
#ID
df["ID"] = df['ID'].str.split(' ').str[0]
df["ID"]

0        Secret
1        Public
2        Public
3        Public
4            No
          ...  
16371        No
16372        No
16373    Secret
16374    Secret
16375       NaN
Name: ID, Length: 16376, dtype: object

In [3757]:
df["ID"] = df["ID"].replace(["No"], "None")
df["ID"]

0        Secret
1        Public
2        Public
3        Public
4          None
          ...  
16371      None
16372      None
16373    Secret
16374    Secret
16375       NaN
Name: ID, Length: 16376, dtype: object

* Some data is in the wrong format. For example, Year and APPEARANCES are floats but they really should be ints (as you cannot have a non-int number of appearances).

In [3758]:
#To convert to int
df['Year'] = df['Year'].astype('Int64')
df['Year']

0        1962
1        1941
2        1974
3        1963
4        1950
         ... 
16371    <NA>
16372    <NA>
16373    <NA>
16374    <NA>
16375    <NA>
Name: Year, Length: 16376, dtype: Int64

In [3759]:
df["APPEARANCES"]= df["APPEARANCES"].astype('Int64')  

In [3760]:
#FIRST APPEARANCE -- All record are matching with Year
all(
df[(df['FIRST APPEARANCE'].str.split('-',1).str[1] == df["Year"].astype(str).str[2:]) 
& (df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')
& (df["Year"].notnull()) & (df["Year"]!=u'')].count()
    )

True

In [3761]:
#Same number on both sides not null and equal. See above.
df[(df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')
& (df["Year"].notnull()) & (df["Year"]!=u'')][["FIRST APPEARANCE", "Year"]].count()

FIRST APPEARANCE    15561
Year                15561
dtype: int64

In [3762]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   page_id           16376 non-null  int64 
 1   name              16376 non-null  object
 2   SecretIdentity    16344 non-null  object
 3   origin            2362 non-null   object
 4   urlslug           16376 non-null  object
 5   ID                12606 non-null  object
 6   ALIGN             13564 non-null  object
 7   EYE               6609 non-null   object
 8   HAIR              12112 non-null  object
 9   SEX               15522 non-null  object
 10  GSM               90 non-null     object
 11  ALIVE             16373 non-null  object
 12  APPEARANCES       15280 non-null  Int64 
 13  FIRST APPEARANCE  15561 non-null  object
 14  Year              15561 non-null  Int64 
dtypes: Int64(2), int64(1), object(12)
memory usage: 1.9+ MB


In [3763]:
df.duplicated()#There is not duplicates according to this function

0        False
1        False
2        False
3        False
4        False
         ...  
16371    False
16372    False
16373    False
16374    False
16375    False
Length: 16376, dtype: bool

* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.


In [3764]:
df["ID"] = df["ID"].fillna("None")
df["ALIGN"] = df["ALIGN"].fillna("None") 
df["origin"] = df["origin"].fillna("NA") 
df["EYE"] = df["EYE"].fillna("NA") 
df["HAIR"] = df["HAIR"].fillna("NA") 
df["APPEARANCES"] = df["APPEARANCES"].fillna(0)
df["SEX"] = df["SEX"].fillna("NA") 
df["ALIVE"] = df["ALIVE"].fillna("NA") 
df["FIRST APPEARANCE"] = df["FIRST APPEARANCE"].fillna(0)
df["Year"] = df["Year"].fillna(0)




In [3765]:
del df["GSM"]

In [3766]:
df.columns = map(str.lower, df.columns)

In [3767]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   page_id           16376 non-null  int64 
 1   name              16376 non-null  object
 2   secretidentity    16344 non-null  object
 3   origin            16376 non-null  object
 4   urlslug           16376 non-null  object
 5   id                16376 non-null  object
 6   align             16376 non-null  object
 7   eye               16376 non-null  object
 8   hair              16376 non-null  object
 9   sex               16376 non-null  object
 10  alive             16376 non-null  object
 11  appearances       16376 non-null  Int64 
 12  first appearance  16376 non-null  object
 13  year              16376 non-null  Int64 
dtypes: Int64(2), int64(1), object(11)
memory usage: 1.8+ MB


In [3768]:
df["origin"].isna().count()
#df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')

16376

In [3769]:
df[df["origin"] != 'NA']

,page_id,name,secretidentity,origin,urlslug,id,align,eye,hair,sex,alive,appearances,first appearance,year
38,1677,Rogue,Anna Marie,Earth-616,http://marvel.fandom.com/wiki/Rogue_(Anna_Marie)_(Earth-616),Secret,Good,Green,Auburn,Female,Living Characters,850,0,0
60,1836,Human Torch,Android,Earth-616,http://marvel.fandom.com/wiki/Human_Torch_(Android)_(Earth-616),Public,Good,Blue,Blond,Male,Living Characters,548,Oct-39,1939
106,18186,Venom,Symbiote,Earth-616,http://marvel.fandom.com/wiki/Venom_(Symbiote)_(Earth-616),Known,Neutral,Variable,No,Agender,Living Characters,348,May-84,1984
138,1256,Bullseye,Lester,Earth-616,http://marvel.fandom.com/wiki/Bullseye_(Lester)_(Earth-616),Secret,Bad,Blue,Blond,Male,Living Characters,277,Mar-76,1976
148,2511,Peter Parker,Ben Reilly,Earth-616,http://marvel.fandom.com/wiki/Peter_Parker_(Ben_Reilly)_(Earth-616),Secret,Good,Hazel,Blond,Male,Deceased Characters,263,Oct-75,1975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16367,694583,Marcus,Kate Bishop's neighbor,Earth-616,http://marvel.fandom.com/wiki/Marcus_(Kate_Bishop%27s_neighbor)_(Earth-616),Public,Good,Hazel,Bald,Male,Living Characters,0,0,0
16368,703892,Marcy,Offer's employee,Earth-616,http://marvel.fandom.com/wiki/Marcy_(Offer%27s_employee)_(Earth-616),Public,Neutral,NA,Brown,Female,Living Characters,0,0,0
16372,665474,Thane,Thanos' son,Earth-616,http://marvel.fandom.com/wiki/Thane_(Thanos%27_son)_(Earth-616),None,Good,Blue,Bald,Male,Living Characters,0,0,0
16373,695217,Tinkerer,Skrull,Earth-616,http://marvel.fandom.com/wiki/Tinkerer_(Skrull)_(Earth-616),Secret,Bad,Black,Bald,Male,Living Characters,0,0,0


In [3770]:
see_possible_categories("id")

,page_id,appearances,year
id,,,
Known,785553,1116,29893
None,1770871574,56075,10323581
Public,1613083818,100208,8657706
Secret,1531859636,102871,11876655
